# Introduction 

In the following notebook, I will be cleaning the energy_dataset.csv file located [here](https://github.com/KishenSharma6/Weather-Energy-Consumption-in-Spain/tree/master/Data/01_Raw_Data)

**Read in libraries for notebook**

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Set notebook preferences**

In [15]:
#Set preferences for pandas 
pd.set_option("display.max_columns", 101)

#Set style for visualizations
plt.style.use('Solarize_Light2')

**Read in data**

In [16]:
#Set path to raw data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Spain Hourly Energy Demand and Weather'

#Read in raw data
df = pd.read_csv(path + '/Data/01_Raw_Data/energy_dataset.csv')

# Data Overview

* time: Datetime index localized to CET
* generation biomass: biomass generation in MW
* generation fossil brown coal/lignite: coal/lignite generation in MW
* generation fossil coal-derived gas: coal gas generation in MW
* generation fossil gas: gas generation in MW
* generation fossil hard coal: coal generation in MW
* generation fossil oil: oil generation in MW
* generation fossil oil shale: shale oil generation in MW
* generation fossil peat: peat generation in MW
* generation geothermal: geothermal generation in MW
* generation hydro pumped storage aggregated: hydro1 generation in MW
* generation hydro pumped storage consumption: hydro2 generation in MW
* generation hydro run-of-river and poundage: hydro3 generation in MW
* generation hydro water reservoir: hydro4 generation in MW
* generation marine: sea generation in MW
* generation nuclear: nuclear generation in MW
* generation other: other generation in MW
* generation other renewable: other renewable generation in MW
* generation solar: solar generation in MW
* generation waste: waste generation in MW
* generation wind offshore: wind offshore generation in MW
* generation wind onshore: wind onshore generation in MW
* forecast solar day ahead: forecasted solar generation
* forecast wind offshore eday ahead: forecasted offshore wind generation
* forecast wind onshore day ahead: forecasted onshore wind generation
* total load forecast: forecasted electrical demand
* total load actual: actual electrical demand
* price day ahead: forecasted price EUR/MWh
* price actual: price in EUR/MWh

**Data Preview**

In [17]:
#Print df shape
print('Shape of data:', df.shape)

#View head
df.head()

Shape of data: (35064, 29)


,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,863.0,1051.0,1899.0,0.0,7096.0,43.0,73.0,49.0,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,920.0,1009.0,1658.0,0.0,7096.0,43.0,71.0,50.0,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,1164.0,973.0,1371.0,0.0,7099.0,43.0,73.0,50.0,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,1503.0,949.0,779.0,0.0,7098.0,43.0,75.0,50.0,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,1826.0,953.0,720.0,0.0,7097.0,43.0,74.0,42.0,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


# Data Cleaning

**Drop columns not pertaining to price and load forecasts**

Also removing spaces in remaining column headers

In [18]:
#Subset price and load columns
df = df[['time', 'total load forecast', 'total load actual', 'price day ahead', 'price actual']]

#Replace \s with _ in column headers
df.columns = df.columns.str.replace('\s', '_', regex = True)

#Check
df.head()

,time,total_load_forecast,total_load_actual,price_day_ahead,price_actual
0,2015-01-01 00:00:00+01:00,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,21785.0,20264.0,38.41,56.04


**Missing Values**

In [19]:
#Check for missing values
df.isna().sum()

time                    0
total_load_forecast     0
total_load_actual      36
price_day_ahead         0
price_actual            0
dtype: int64

In [20]:
#Capture rows with at least one missing value and assign to missing
missing = df[df.isnull().any(axis=1)]

#View missing and number of rows with missing data
print('Number of missing rows: {}\nTotal % missing from data {}%'.format(len(missing),round(len(missing)/len(df),4) * 100))
display(missing)

Number of missing rows: 36
Total % missing from data 0.1%


,time,total_load_forecast,total_load_actual,price_day_ahead,price_actual
108,2015-01-05 12:00:00+01:00,23209.0,NaN,35.50,79.14
109,2015-01-05 13:00:00+01:00,23725.0,NaN,36.80,73.95
110,2015-01-05 14:00:00+01:00,23614.0,NaN,32.50,71.93
111,2015-01-05 15:00:00+01:00,22381.0,NaN,30.00,71.50
112,2015-01-05 16:00:00+01:00,21371.0,NaN,30.00,71.85
113,2015-01-05 17:00:00+01:00,20760.0,NaN,30.60,80.53
661,2015-01-28 13:00:00+01:00,36239.0,NaN,65.00,77.62
751,2015-02-01 07:00:00+01:00,24379.0,NaN,56.10,16.98
752,2015-02-01 08:00:00+01:00,27389.0,NaN,57.69,19.56
753,2015-02-01 09:00:00+01:00,30619.0,NaN,60.01,23.13


*Resolve Missing Values*

In [21]:
#Apply forward and back fill
df = df.ffill().bfill()

#Check
df[df.isnull().any(axis=1)]

,time,total_load_forecast,total_load_actual,price_day_ahead,price_actual


**Clean and extract time data**

In [22]:
#Remove +01:00
df.time.replace('[+].*','', inplace=True,regex=True)

#Rename dt_iso to time and set type
df.rename(columns = {'time':'date_time'}, inplace = True)
df['date_time'] = pd.to_datetime(df['date_time'])

#Create columns extracting time, month, date, year data
df['date'] = df.date_time.dt.date
df['time'] = df.date_time.dt.time
df['weekday'] = df.date_time.dt.day_name()
df['month'] = df.date_time.dt.month_name()
df['year'] = df.date_time.dt.year

#Set date_time as index
df.set_index('date_time', inplace=True)

#Check
display(df.head(3))

,total_load_forecast,total_load_actual,price_day_ahead,price_actual,date,time,weekday,month,year
date_time,,,,,,,,,
2015-01-01 00:00:00,26118.0,25385.0,50.10,65.41,2015-01-01,00:00:00,Thursday,January,2015
2015-01-01 01:00:00,24934.0,24382.0,48.10,64.92,2015-01-01,01:00:00,Thursday,January,2015
2015-01-01 02:00:00,23515.0,22734.0,47.33,64.48,2015-01-01,02:00:00,Thursday,January,2015


# Write file to CSV

In [23]:
#Print shape
print('Cleaned data shape:', df.shape)

#Write to CSV
df.to_csv(path + '/Data/02_Cleaned_Data/2020_0505_Cleaned_Energy_Dataset.csv')

Cleaned data shape: (35064, 9)
